In [1]:
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt  # data visualization
import matplotlib.dates as mdates
import warnings
from dateutil.relativedelta import relativedelta
from functions import *
import seaborn as sns

warnings.filterwarnings(action='ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

showGraphs = True

%matplotlib inline


In [2]:
# Any results you write to the current directory are saved as output.
coinbase = pd.read_csv(
    "./data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")

coinbase.describe()
coinbase.tail()
coinbase.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4857377 entries, 0 to 4857376
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Timestamp          int64  
 1   Open               float64
 2   High               float64
 3   Low                float64
 4   Close              float64
 5   Volume_(BTC)       float64
 6   Volume_(Currency)  float64
 7   Weighted_Price     float64
dtypes: float64(7), int64(1)
memory usage: 296.5 MB


Filtrar intervalo de dados

In [7]:
filtered = filterByInterval(coinbase, relativedelta(months=15))

if hasMissingData(filtered['Timestamp'].values):
    raise Exception("Missing data in the dataframe")

print("O intervalo entre datas é de {} até {}, somando um total de {} registros.".format(
    getFirstTimestamp(filtered), getLastTimestamp(filtered), filtered.shape[0]))


O intervalo entre datas é de 31/12/2019 00:00:00 até 31/12/2020 00:00:00, somando um total de 527041 registros.


Prepara os dados históricos agrupandos por hora

In [4]:
historical = filtered.dropna().reset_index(drop=True)
historical.Timestamp = pd.to_datetime(historical.Timestamp, unit='s')
# historical['date'] = historical.Timestamp.dt.date
historical['dateHour'] = historical.Timestamp.dt.strftime('%Y-%m-%d %H')
historical_per_hour = historical.groupby(historical.dateHour).mean()

In [5]:
if showGraphs:
    kw = dict(annot_yaxis=10000, annot_xaxis=-1000, annot_xaxis_pos=-6000, annot_yaxis_pos=-
              8000, angle="angle,angleA=0,angleB=90", angle_pos="angle,angleA=0,angleB=-90")
    # figureCloses(historical_per_hour, 'Close', **kw)


TypeError: an integer is required (got type str)

<Figure size 1008x432 with 0 Axes>

Gráfico Pairplot

In [ ]:
if showGraphs:
    plt.figure(figsize = (30,30))
    sns.pairplot(historical_per_hour)


Preparação dos Dados

In [10]:
data = historical_per_hour.Close
data = data.values
max = np.max(data)
data = data/max

8781


Separar dados de Treino e Teste

In [ ]:
X = [data[i:i+10] for i in range(len(data)-11)]
Y = [data[i+10] for i in range(len(data)-11)]
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, shuffle=False)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

Criação do Modelo

In [ ]:
np.random.seed(0)

lstm = LSTM(64, activation='relu')

model = Sequential([
    Bidirectional(lstm, input_shape=(1, 10)),
    Dropout(0.15),
    Dense(1)
])
model.compile(loss='mse', optimizer='adam')
history = model.fit(X_train, Y_train, epochs=20)


Treinamento do Modelo

In [ ]:
pred_train = model.predict(X_train)

print("Mean absolute error regression loss: %.4f" %
      mean_absolute_error(Y_train, pred_train))
print("Root mean squared error regression loss: %.4f" %
      np.sqrt(mean_squared_error(Y_train, pred_train)))
print("R2 score: %.4f" % r2_score(Y_train, pred_train))


Gráfico da função de perda

In [ ]:
if showGraphs:
    plt.figure(figsize=(8, 8))
    plt.title("Loss evolution")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.plot(history.history['loss'])

Gráfico da predição comparado com os dados de teste

In [ ]:
if showGraphs:
    preds1 = pred_train * max
    plt.figure(figsize=(12, 12))
    plt.plot(Y_train * max, c='red', label='predicted values', linewidth=2)
    plt.plot(preds1, c='blue', label='real values', linewidth=2)
    plt.xlabel("Days")
    plt.title("prediction on train data")
    plt.legend()


Gráfico da predição comparado com os dados de validação

In [ ]:
if showGraphs:
    preds = model.predict(X_test) * max
    plt.figure(figsize=(14, 14))
    plt.plot(preds, c='red', label='predicted values', linewidth=2)
    plt.plot(Y_test * max, c='blue', label='real values', linewidth=2)
    plt.xlabel("Days")
    plt.title("prediction on test data")
    plt.legend()


Train using LSTM 

In [ ]:
# # The LSTM architecture
# regressor = Sequential()
# # First LSTM layer with Dropout regularisation
# regressor.add(LSTM(units=50, return_sequences=True,
#               input_shape=(X_train.shape[1], 1)))
# regressor.add(Dropout(0.2))

# # Second LSTM layer
# regressor.add(LSTM(units=50, return_sequences=True))
# regressor.add(Dropout(0.2))

# # Third LSTM layer
# regressor.add(LSTM(units=50, return_sequences=True))
# regressor.add(Dropout(0.5))

# # Fourth LSTM layer
# regressor.add(LSTM(units=50))
# regressor.add(Dropout(0.5))

# # The output layer
# regressor.add(Dense(units=1))

# # Compiling the RNN
# regressor.compile(optimizer='adam', loss='mean_absolute_error')
# # Fitting to the training set
# regressor.fit(X_train, y_train, epochs=1, batch_size=500)


This is clearly overfitting